# Linode 

> Dealing with my preferred cloud provider.

In [18]:
# | default_exp linode
# | export
import os
import requests

from functools import cached_property

In [19]:
# | export


class LinodeClient:
    """
    A client instance that facilitates communication with the Linode API.
    """

    def __init__(self, token):
        self.base_url = "https://api.linode.com/v4"
        self.session = requests.Session()
        self.session.headers.update(
            {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {token}",
            }
        )

    def api_call(self, method, endpoint, data=None):
        url = f"{self.base_url}{endpoint}"
        response = self.session.request(method, url, json=data)
        return response

    @cached_property
    def firewall_id(self, label="tjcloud"):
        firewalls = self.api_call("GET", "/networking/firewalls").json()["data"]
        firewall = list(filter(lambda f: f["label"] == label, firewalls))[0]
        return firewall["id"]

    def enable_firewall(self, status="enabled"):
        self.api_call(
            "PUT", f"/networking/firewalls/{self.firewall_id}", data={"status": status}
        )

    def disable_firewall(self):
        self.enable_firewall(status="disabled")

    @property
    def is_firewall_enabled(self):
        firewall = self.api_call(
            "GET", f"/networking/firewalls/{self.firewall_id}"
        ).json()
        return firewall["status"] == "enabled"

## Firewall

We create a mechanism to control a firewall that is created for the Kubernetes cluster. Normally, it only allos web traffic but may allow other traffic depending on the needs of the cluster. 

In [22]:
# | eval: false
client = LinodeClient(os.getenv("LINODE_TOKEN"))
print(client.is_firewall_enabled)
client.enable_firewall()
print(client.is_firewall_enabled)
client.disable_firewall()
print(client.is_firewall_enabled)

True
True
False
